In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode, pearsonr
import csv
import matplotlib.dates
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from datetime import *

%matplotlib inline
# -*-coding:utf-8-*-

In [126]:
# 导入数据
df = pd.read_csv('./data/round1_ijcai_18_train_20180301.txt', sep=' ')
df = df.applymap(lambda x: np.nan if (x==-1)|(x=='-1') else x)
dfA = pd.read_csv('./data/round1_ijcai_18_test_a_20180301.txt', sep=' ')
dfA = dfA.applymap(lambda x: np.nan if (x==-1)|(x=='-1') else x)
dfCommitA = pd.read_csv('./data/round1_ijcai_18_result_demo_20180301.txt', sep=' ')

print(df.info())
print(dfA.info())
print(dfCommitA.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478138 entries, 0 to 478137
Data columns (total 27 columns):
instance_id                  478138 non-null int64
item_id                      478138 non-null int64
item_category_list           478138 non-null object
item_property_list           478138 non-null object
item_brand_id                477665 non-null float64
item_city_id                 477861 non-null float64
item_price_level             478138 non-null int64
item_sales_level             477225 non-null float64
item_collected_level         478138 non-null int64
item_pv_level                478138 non-null int64
user_id                      478138 non-null int64
user_gender_id               465236 non-null float64
user_age_level               477174 non-null float64
user_occupation_id           477174 non-null float64
user_star_level              477174 non-null float64
context_id                   478138 non-null int64
context_timestamp            478138 non-null int64
contex

In [29]:
# 查看整个训练集的重复情况
print('重复行数：', len(df[df.duplicated(subset=df.columns[:-1])]))
print(df[df.duplicated()])
'''
抛开样本id不说，重复的数据可能仅仅是重复了，也可能代表同样的广告点击事件在不同时间发生了多次
不过并没有出现相同特征不同标签的数据，因此暂时按去重方式处理
'''

重复行数： 27
                instance_id              item_id  \
5484    5089270100365094282   347767868088948904   
7020    4984307821968110563   509936336103770855   
17149   3370133319712542740  7183501913997303466   
17215   3370133319712542740  7183501913997303466   
59598   7997153237845815308  8146740840340407990   
59804   8775022180950882358  8012757300722536153   
61595   7448305858307931414  6873423853724036003   
84828   7490973479247675308   509936336103770855   
87852    683023144061510567  4627333961840448559   
89214   1862005078700087859  7204670103692122629   
95550    261667037721904400   823401505312413533   
152263   899927513310799701  6042980124947456838   
181492  5547571229532052578  6872176842641369487   
186346  7777421848532810323  8021382380127415282   
200793  4713809052247380232  7750399525178174489   
231094   833314485724646554   365202798058356244   
237497  7983370106370683357  3522778966304956906   
269571  3779422444610157507  3058758668824976333   
293

'抛开样本id不说，重复的数据可能仅仅是重复了，也可能代表同样的广告点击事件在不同时间发生了多次'

In [127]:
# 去重并将上下文属性字段处理成键值对列表格式
startTime = datetime.now()
df.drop_duplicates(inplace=True)
df['predict_category_property'] = df[df.predict_category_property.notnull()]['predict_category_property'].map(
    lambda x: {kv.split(':')[0]:(kv.split(':')[1].split(',') if kv.split(':')[1]!='-1' else []) for kv in x.split(';')})
print('cost time: ', datetime.now() - startTime)
print(df['predict_category_property'].head())
print(df.info())

cost time:  0:00:04.453700
0    {'5799347067982556520': [], '50966009553013476...
1    {'5799347067982556520': ['9172976955054793469'...
2    {'5799347067982556520': ['5131280576272319091'...
3    {'509660095530134768': [], '579934706798255652...
4    {'5799347067982556520': ['9172976955054793469'...
Name: predict_category_property, dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 478111 entries, 0 to 478137
Data columns (total 27 columns):
instance_id                  478111 non-null int64
item_id                      478111 non-null int64
item_category_list           478111 non-null object
item_property_list           478111 non-null object
item_brand_id                477638 non-null float64
item_city_id                 477834 non-null float64
item_price_level             478111 non-null int64
item_sales_level             477198 non-null float64
item_collected_level         478111 non-null int64
item_pv_level                478111 non-null int64
user_id               

In [31]:
# 观察训练集和测试集中商品/用户/上下文/店铺的重叠情况
for x in ['instance_id','item_id','user_id','context_id','shop_id']:
    dfASet = set(dfA[x].values)
    dfSet = set(df[x].values)
    print('%s count in df: '%x, len(dfSet))
    print('%s count in dfA: '%x, len(dfASet))
    print('diff %s in dfA: '%x, len(dfASet - dfSet))
'''
样本id有3条重复，且对比发现训练集的样本id也不唯一。这有点奇怪，后面再查探下情况
商品大部分都是训练集有的
用户只有小部分相同
上下文则完全不同
'''

instance_id count in df:  478087
instance_id count in dfA:  18371
diff instance_id in dfA:  18368
item_id count in df:  10075
item_id count in dfA:  3695
diff item_id in dfA:  161
user_id count in df:  197694
user_id count in dfA:  13573
diff user_id in dfA:  9947
context_id count in df:  478111
context_id count in dfA:  18371
diff context_id in dfA:  18371
shop_id count in df:  3959
shop_id count in dfA:  2015
diff shop_id in dfA:  44


'\n样本id有3条重复，且对比发现训练集的样本id也不唯一。这有点奇怪，后面再查探下情况\n商品大部分都是训练集有的\n用户只有小部分相同\n上下文则完全不同\n'

In [32]:
# 研究相同样本id的数据情况
for x in ['instance_id']:
    sameId = df[df.duplicated(subset=[x])][x].values
    print(df[df[x].isin(sameId)].sort_index(by=[x])[['instance_id','item_id','user_id','context_id','shop_id']])
'''
重复样本id的数据似乎没有什么共同点，暂且忽略当作不同样本处理
'''

                instance_id              item_id              user_id  \
356659   191012710137081307  3077296203572359783  5325662788038268161   
346190   191012710137081307  7084821226558739429  9122702826826959749   
88934    251764820957840061  1641974005505706555  8476098988805147931   
403424   251764820957840061  2606171203888063182  4314800938910797474   
108935   348545486282294646  3283029658215624826  1874662220027116536   
129013   348545486282294646  5322956081233923609  5394255053178051804   
123957   644690756602987584  3275099097645146109  7235904247809926126   
253167   644690756602987584   696234301202309885  5182288156881045163   
443738  1037061371711078396   124537517574802912  6149009034308291860   
448702  1037061371711078396   379545858977092666  5385170184093781769   
327627  2264716673495444735  8568101554277882193  1520509896533577843   
306198  2264716673495444735   953306439453034088  4289827392090131624   
431441  2535273059737073357  1383886353886782040  1

/home/qy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  after removing the cwd from sys.path.


In [34]:
# 研究测试集中与训练集相同样本id的数据
sameIds = dfA[dfA.instance_id.isin(df.instance_id)].instance_id.values
sameDf = dfA[dfA.instance_id.isin(sameIds)]
sameDf.set_index(['instance_id'], inplace=True)
for col in dfA.columns[1:]:
    sameDf[col] = sameDf[col].values == df[df.instance_id.isin(sameIds)][col].values
print(sameDf[['item_id','user_id','context_id','shop_id']])
'''虽然测试集中有三个样本编号与训练集相同，但是样本内容却相差甚远，暂时当作不相关的样本处理'''

/home/qy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


                     item_id  user_id  context_id  shop_id
instance_id                                               
8860419674042916065    False    False       False    False
4773277620195885623    False    False       False    False
892803262387109244     False    False       False    False


'虽然测试集中有三个样本编号与训练集相同，但是样本内容却相差甚远'

In [84]:
# 商品数据集
startTime = datetime.now()
itemDf = df.drop_duplicates(subset=['item_id'])[[
    'item_id', 'item_category_list', 'item_property_list','item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level','item_collected_level', 'item_pv_level',
    'shop_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description'
]]
itemDf['item_category_list'] = itemDf['item_category_list'].map(lambda x: x.split(';'))
itemDf['item_category0'] = itemDf['item_category_list'].map(lambda x: x[0])
itemDf['item_category1'] = itemDf['item_category_list'].map(lambda x: x[1] if len(x)>1 else np.nan)
itemDf['item_category2'] = itemDf['item_category_list'].map(lambda x: x[2] if len(x)>2 else np.nan)
itemDf['item_property_list'] = itemDf['item_property_list'].map(lambda x: x.split(';'))
tempDf = pd.pivot_table(df, index=['item_id'], values=['instance_id','is_trade'], aggfunc={'instance_id':len,'is_trade':[np.sum,np.average]})
tempDf.columns = ['times','trade_ratio','trade_times']
itemDf = itemDf.merge(tempDf, how='left', left_on='item_id', right_index=True)
print('cost time: ', datetime.now() - startTime)
print(itemDf.head())
print(itemDf.describe())

cost time:  0:00:00.361807
                item_id                          item_category_list  \
0   3412720377098676069  [7908382889764677758, 5799347067982556520]   
6    285660928590172217  [7908382889764677758, 8277336076276184272]   
8   5202355029344881809  [7908382889764677758, 5755694407684602296]   
29  4596211878136725009  [7908382889764677758, 4879721024980945592]   
30  8626373102868398325  [7908382889764677758, 8277336076276184272]   

                                   item_property_list  item_brand_id  \
0   [2072967855524022579, 5131280576272319091, 263...   1.975590e+18   
6   [2072967855524022579, 5131280576272319091, 263...   9.057103e+18   
8   [2072967855524022579, 5131280576272319091, 263...   5.520679e+18   
29  [2072967855524022579, 2636395404473730413, 462...   2.768135e+18   
30  [2878948665402366264, 2636395404473730413, 372...   6.789334e+18   

    item_city_id  item_price_level  item_sales_level  item_collected_level  \
0   3.948283e+18                 3 

In [130]:
# 研究商品类目分布
print(itemDf.item_category0.value_counts())
print(itemDf.item_category1.value_counts())
print(itemDf.item_category2.value_counts())

7908382889764677758    10075
Name: item_category0, dtype: int64
8277336076276184272    2606
5755694407684602296    2375
5799347067982556520    1478
509660095530134768     1153
2011981573061447208     735
7258015885215914736     674
8710739180200009128     367
3203673979138763595     200
4879721024980945592     179
2436715285093487584     150
2642175453151805566      95
1968056100269760729      42
22731265849056483        21
Name: item_category1, dtype: int64
8868887661186419229    82
6233669177166538628    13
Name: item_category2, dtype: int64


In [56]:
# 用户数据集
startTime = datetime.now()
userDf = df.drop_duplicates(subset=['user_id'])[[
    'user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level'
]]
tempDf = pd.pivot_table(df, index=['user_id'], values=['instance_id','is_trade'], aggfunc={'instance_id':len,'is_trade':[np.sum,np.average]})
tempDf.columns = ['times','trade_ratio','trade_times']
userDf = userDf.merge(tempDf, how='left', left_on='user_id', right_index=True)
print('cost time: ', datetime.now() - startTime)
print(userDf.head())
print(userDf.describe())

cost time:  0:00:04.411891
               user_id  user_gender_id  user_age_level  user_occupation_id  \
0  4505772604969228686             1.0          1003.0              2005.0   
1  2692638157208937547             0.0          1002.0              2005.0   
2  5247924392014515924             0.0          1003.0              2005.0   
3  2681414445369714628             1.0          1004.0              2005.0   
4  2729475788342039013             0.0          1002.0              2005.0   

   user_star_level  times  trade_ratio  trade_times  
0           3003.0      6          0.0            0  
1           3006.0     10          0.0            0  
2           3004.0      2          0.0            0  
3           3006.0      1          0.0            0  
4           3001.0      2          0.0            0  
            user_id  user_gender_id  user_age_level  user_occupation_id  \
count  1.976940e+05   192189.000000   197255.000000       197255.000000   
mean   4.610097e+18        0.2

In [68]:
# 店铺数据集
startTime = datetime.now()
shopDf = df.drop_duplicates(subset=['shop_id'])[[
    'shop_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description'
]]
shopDf['item_num'] = shopDf.shop_id.map(lambda x: len(np.unique(df[df.shop_id==x]['item_id'])))
tempDf = pd.pivot_table(df, index=['shop_id'], values=['instance_id','is_trade'], aggfunc={'instance_id':len,'is_trade':[np.sum,np.average]})
tempDf.columns = ['times','trade_ratio','trade_times']
shopDf = shopDf.merge(tempDf, how='left', left_on='shop_id', right_index=True)
print('cost time: ', datetime.now() - startTime)
print(shopDf.head())
print(shopDf.describe())

cost time:  0:00:03.028228
                shop_id  shop_review_num_level  shop_review_positive_rate  \
0   6765930309048922341                      4                   1.000000   
6   4885989684392199728                     15                   0.985427   
29  6768355095170828465                     14                   0.980592   
36  4691269887550643960                     12                   0.967752   
40  6597981382309269962                     21                   0.997040   

    shop_star_level  shop_score_service  shop_score_delivery  \
0              5002            1.000000             1.000000   
6              5012            0.974878             0.976863   
29             5012            0.972680             0.972113   
36             5010            0.974225             0.971071   
40             5018            0.979661             0.979589   

    shop_score_description  item_num  times  trade_ratio  trade_times  
0                 1.000000         1     20     0.000

In [75]:
# 相同用户和商品之间的事件数据集
startTime = datetime.now()
userItemDf = pd.pivot_table(df[df.duplicated(subset=['user_id','item_id'], keep=False)], index=['user_id','item_id'], values='is_trade', aggfunc=[len, np.sum])
userItemDf.columns=['times', 'trade_times']
userItemDf.reset_index(inplace=True)
print('cost time: ', datetime.now() - startTime)
print(userItemDf.head())
print(userItemDf.describe())

cost time:  0:00:00.180822
            user_id              item_id  times  trade_times
0    24779788309075  5649087492658319596      2            0
1   810869007662288  3129052219744291798      2            0
2   867581361033096   844384142909881247      3            0
3  1258438522225233  4869448032320184700      2            0
4  1258438522225233  5087145291415594054      2            0
            user_id       item_id         times   trade_times
count  2.824800e+04  2.824800e+04  28248.000000  28248.000000
mean   4.599906e+18  4.600340e+18      2.198457      0.050411
std    2.665603e+18  2.718791e+18      0.577773      0.221528
min    2.477979e+13  1.097631e+15      2.000000      0.000000
25%    2.307407e+18  2.251088e+18      2.000000      0.000000
50%    4.582784e+18  4.684771e+18      2.000000      0.000000
75%    6.900328e+18  7.014009e+18      2.000000      0.000000
max    9.223335e+18  9.221980e+18     15.000000      2.000000


In [138]:
# 观察有重复点击的事件下单率
print(df.is_trade.mean())
print(userItemDf.trade_times.value_counts())
'''这么算算，好像重复点击查看的广告下单率要更高一些呢'''

0.018811531213462983
0    26841
1     1390
2       17
Name: trade_times, dtype: int64


In [141]:
# 检查测试集中与训练集同用户同商品的点击事件数目
df['union_index'] = df.apply(lambda x: '%s_%s'%(x.user_id, x.item_id), axis=1)
dfA['union_index'] = dfA.apply(lambda x: '%s_%s'%(x.user_id, x.item_id), axis=1)
print(dfA[dfA.union_index.isin(df.union_index)])
'''测试集中与训练集重复联合id的这部分事件，是否会更可能发生购买行为？'''

               instance_id              item_id  \
37     4196377277000518389  2349124492995958413   
48     3090692985891227788   919980016657888153   
130    5585024284234417351  8629327205974770664   
146    8614041918260246627   919980016657888153   
170    6426031625233383026  8629327205974770664   
193     952160001342209502   919980016657888153   
194    6001454270979878025  2349124492995958413   
240    7094087069206466771  2349124492995958413   
263    7433161595507622639   557883074900282934   
284    8907543397538166800  7252664911628293109   
332    4405636244411148307   919980016657888153   
334    1210139528085356394  2349124492995958413   
348    8269745254365565671  8629327205974770664   
404    3737231766433875650  2349124492995958413   
419     912880647279198383  8574382974114632001   
441    7285031570611510992  1519543411555287447   
464     549695153720110738   919980016657888153   
474    5672469633587645992  2127588125899824865   
491    5944696426464716476  862